# Dimensional Analysis

This notebook prototypes a small controlled natural language for specifying physical properties.
Our example sentence is

    the ball has a mass of 5 kg and a kinetic energy of 12 mN

and we want to obtain the logical expression

    (mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 meter Newton)).

However, we have to use [dimensional analysis](https://en.wikipedia.org/wiki/Dimensional_analysis)
to disambiguate whether *12 mN* stands for *12 milli Newton* or for *12 meter Newton*.
Since *milli Newton* is not an energy, we can discard it.

This example was created for a paper submitted to FCR 2020.

In [1]:
subdir dim_analysis

Changed to subdirectory dim_analysis

Successfully reloaded GF

### Grammar

In [2]:
abstract Grammar = {
    cat
        Object;
        Measurable;   -- kinetic energy
        Measurement;  -- a kinetic energy of 3 m N
        Unit;         -- m N
        S;            -- sentence
        
    fun
        hasProp : Object -> Measurement -> S;
        unitCombine : Unit -> Unit -> Unit;
        objectCombine : Object -> Object -> Object;
        
        measure : Measurable -> Int -> Unit -> Measurement;
        combine : Measurement -> Measurement -> Measurement;
        
        theball : Object;
        thetrain : Object;
        
        eKin : Measurable;
        mass : Measurable;
        
        meter : Unit;
        newton : Unit;
        gram : Unit;
        
        milli : Unit -> Unit;
        kilo : Unit -> Unit;
}

Defined Grammar

In [3]:
concrete GrammarEng of Grammar = {
  param
    Number = Sg | Pl;
  oper
    have : Number => Str = table {Sg => "has"; Pl => "have"};
  lincat
    Object = {s : Str; n : Number};
    Measurable = Str;
    Measurement = Str;
    Unit = Str;
    S = Str;
    
  lin
    hasProp obj m = obj.s ++ have ! obj.n ++ m;
    unitCombine a b = a ++ b;
    objectCombine a b = {s = a.s ++ "and" ++ b.s; n = Pl};
    measure m int unit = "a" ++ m ++ "of" ++ int.s ++ unit;
    combine a b = a ++ "and" ++ b;
    theball = {s = "the ball"; n = Sg};
    thetrain = {s = "the train"; n = Sg};
    eKin = "kinetic energy";
    mass = "mass";
    meter = "m";
    newton = "N";
    gram = "g";
    milli u = "m" ++ u;
    kilo u = "k" ++ u;
}

In [4]:
generate_random -number=5 | linearize

the train has a mass of 999 m k m

the ball and the train and the ball and the train have a kinetic energy of 999 m

the ball and the train have a mass of 999 N

the train has a kinetic energy of 999 m and a kinetic energy of 999 g and a mass of 999 N

the ball has a mass of 999 N and a mass of 999 m and a kinetic energy of 999 g and a kinetic energy of 999 m

In [5]:
parse "the ball has a mass of 5 k g"

hasProp theball (measure mass 5 (kilo gram))

**The example sentence generates two parse trees**

In [6]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N"

hasProp theball (combine (measure mass 5 (kilo gram)) (measure eKin 12 (milli newton)))

hasProp theball (combine (measure mass 5 (kilo gram)) (measure eKin 12 (unitCombine meter newton)))

In [7]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | vt -nocat | show

2 graphs generated

Dropdown(description='Tree of:', layout=Layout(width='max-content'), options=('hasProp theball (combine (measu…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02=\x00\x00\x01\xbb\x08\x06\x00\x00\x00(\x90$Y\x00\…

### Target Logic and Semantics Construction

In [8]:
theory proplog : ur:?LF =
    proposition : type ❘ # o ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 ❙
    // etc. ❙
❚

Created theory proplog

In [9]:
theory plnq : ur:?LF =
    include ?proplog ❙
    object : type ❘ # ι ❙    // more technical term: individual ❙
❚

Created theory plnq

In [10]:
theory units : ?plnq =
    include ☞http://cds.omdoc.org/urtheories?NatSymbols ❙
    unit  : type      ❘ # u ❙
    mult  : u ⟶ u ⟶ u ❘ # 1 ⋅ 2 ❙   // e.g. meter ⋅ Newton ❙
    udiv  : u ⟶ u ⟶ u ❘ # 1 / 2 ❙
    milli : u ⟶ u     ❘ # milli 1 ❙
    kilo  : u ⟶ u     ❘ # kilo 1 ❙
    
    newton : u ❘ # Newton ❙
    gram   : u ❘ # gram   ❙
    meter  : u ❘ # meter  ❙
    
    quantity : type ❘ # q ❙
    quant : NAT ⟶ u ⟶ q ❙
    ekin : ι ⟶ q ⟶ o ❙
    mass : ι ⟶ q ⟶ o ❙
❚

Created theory units

In [11]:
theory domainTheory : ?plnq =
    include ?units ❙
    ball : ι ❘ # theball ❙
    train : ι ❘ # thetrain ❙
❚

Created theory domainTheory

In [12]:
view SemConstr : http://mathhub.info/comma/jupyter/dim_analysis/Grammar.gf?Grammar -> ?domainTheory =
    NAT = NAT ❙
    zero = zero ❙

    Object = (ι ⟶ o) ⟶ o ❙      // type raised from ι ❙
    Measurable = ι ⟶ q ⟶ o ❙
    Measurement = ι ⟶ o ❙    // unary predicates ❙
    Unit = u ❙
    S = o ❙

    // hasProp : Object ⟶ Measurement ⟶ Sentence ❙
    hasProp = [x, m] (x m) ❙
    // unitCombine : Unit ⟶ Unit ⟶ Unit ❙
    unitCombine = [a,b] a ⋅ b ❙
    //objectCombine : Object ⟶ Object ⟶ Object ❙
    objectCombine = [a, b] [p] (a p) ∧ (b p) ❙
    // measure : Measurable ⟶ Int ⟶ Unit ⟶ Measurement ❙
    measure = [m,n,un] [x] m x (quant n un) ❙
    // combine : Measurement ⟶ Measurement ⟶ Measurement ❙
    combine = [M,N] [x] (M x) ∧ (N x) ❙
    
    theball = [p] p ball ❙
    thetrain = [p] p train ❙
    eKin = ekin ❙
    mass = mass ❙
    meter = meter ❙
    newton = Newton ❙
    gram = gram ❙
    milli = milli ❙
    kilo = kilo ❙
❚

Created view SemConstr

**We get two different logical expressions**

In [13]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | construct -v SemConstr

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 milli Newton))

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 meter⋅Newton))

In [14]:
parse "the ball and the train have a kinetic energy of 12 m N" | construct -v SemConstr

(ekin theball (quant 12 milli Newton))∧(ekin thetrain (quant 12 milli Newton))

(ekin theball (quant 12 meter⋅Newton))∧(ekin thetrain (quant 12 meter⋅Newton))

### Dimensional Analysis with ELPI

Generate the ELPI signature from the MMT theory `domainTheory`.

In [15]:
elpigen -withmeta types domainTheory

Success

In [16]:
elpi: dimCheck
accumulate domainTheory.    % import generated signature

% BASE DIMENSIONS
kind base_dimension type.
type length_dim base_dimension.
type mass_dim base_dimension.
type time_dim base_dimension.

kind dimension type.
% complex dimension, e.g. (cdim [length_dim] [time_dim, time_dim]) for speed
type cdim list base_dimension -> list base_dimension -> dimension.


% removes first occurence
type remove A -> list A -> list A -> prop.
remove X [X|L] L :- !.
remove X [H|L] [H|M] :- remove X L M.

% cancel out equal dimensions
type reduce dimension -> dimension -> prop.
reduce (cdim A B) R :- remove X A A2, remove X B B2, !, reduce (cdim A2 B2) R.
reduce (cdim A B) (cdim A B).

% convert unit to dimension
type utodim unit -> dimension -> prop.
utodim newton (cdim [mass_dim, length_dim] [time_dim, time_dim]).
% utodim joule (cdim [length_dim, mass_dim, length_dim] [time_dim, time_dim]).
utodim gram (cdim [mass_dim] []).
utodim meter (cdim [length_dim] []).
utodim (milli X) Y :- utodim X Y.
utodim (kilo X) Y :- utodim X Y.
utodim (mult A B) (cdim N1 N2) :- utodim A (cdim L1 L2), utodim B (cdim M1 M2), std.append L1 M1 N1, std.append L2 M2 N2.
utodim (udiv A B) (cdim N1 N2) :- utodim A (cdim L1 L2), utodim B (cdim M1 M2), std.append L1 M2 N1, std.append L2 M1 N2.

type dim_eq dimension -> dimension -> prop.
dim_eq (cdim [] []) (cdim [] []).
dim_eq (cdim L LL) (cdim M MM) :- remove X L L2, remove X M M2, dim_eq (cdim L2 LL) (cdim M2 MM).
dim_eq (cdim [] L) (cdim [] M) :- remove X L L2, remove X M M2, dim_eq (cdim [] L2) (cdim [] M2).


% check that quantity has specific dimension.
type dimcheck quantity -> dimension -> prop.
dimcheck (quant _ X) D :-
    utodim X E, 
    reduce D D2,
    reduce E E2,
    dim_eq D2 E2, !.

type check proposition -> prop.
check (and A B)  :- check A, check B.
check (ekin _ Q) :- dimcheck Q (cdim [length_dim, length_dim, mass_dim] [time_dim, time_dim]).
check (mass _ Q) :- dimcheck Q (cdim [mass_dim] []).

Created dimCheck.elpi

**The wrong reading is rejected**

In [17]:
parse "the ball has a kinetic energy of 12 m N"

hasProp theball (measure eKin 12 (milli newton))

hasProp theball (measure eKin 12 (unitCombine meter newton))

In [18]:
parse "the ball has a mass of 5 k g and a kinetic energy of 12 m N" | construct

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 milli Newton))

(mass theball (quant 5 kilo gram))∧(ekin theball (quant 12 meter⋅Newton))

In [19]:
parse "the ball has a kinetic energy of 12 m N" | construct -e | elpi filter dimCheck check

ekin ball (quant 12 (mult meter newton))
